In [78]:
import sys
import numpy as np

In [79]:
def read_off(filename):
    with open(filename) as fn:
        line = fn.readline()
        if (''.join(line.split()) != 'OFF'):
            print(filename + ' is not an OFF file')
            sys.exit(0)
        
        line = fn.readline()
        nvfe = np.asarray(line.split(),dtype=int)
        vertices = np.zeros((nvfe[0],3))
        triangles = np.zeros((nvfe[1],3),dtype=int)
        for i in range(nvfe[0]):
            line = fn.readline()
            vertices[i] = np.asarray(line.split(),dtype=float)
            
        for i in range(nvfe[1]):
            line = fn.readline()
            triangles[i] = np.asarray(line.split(),dtype=int)[1:]
            
        return(vertices,triangles)

In [80]:
lv,lt = read_off('socket.off')

In [81]:
print(lt[:10])

[[  6 398   7]
 [  6 400 398]
 [  6   5 400]
 [400   5 402]
 [403 402 405]
 [ 11 405  12]
 [ 11 403 405]
 [ 11  10 403]
 [403  10 401]
 [400 401 398]]


In [82]:
print(lv[:10])

[[-82.071068   7.071068  25.      ]
 [-75.        10.        25.      ]
 [-67.928932   7.071068  25.      ]
 [-65.         0.        25.      ]
 [-67.928932  -7.071068  25.      ]
 [-75.       -10.        25.      ]
 [-82.071068  -7.071068  25.      ]
 [-85.         0.        25.      ]
 [-82.071068  -7.071068 -25.      ]
 [-75.       -10.       -25.      ]]


In [83]:
def normal(p1,p2,p3):
    U = p2-p1
    V = p3-p1
    N = np.array([U[1]*V[2] - U[2]*V[1], U[2]*V[0] - U[0]*V[2], U[0]*V[1] - U[1]*V[0]])
    if N[2]<0:
        return -N
    else:
        return N

# True if p2 is counterclockwise from p1
def ccw(p1,p2,p3):
    return normal(p1,p2,p3).dot(np.cross(p1-p3,p2-p3))>=0

vertex_to_corner = [set() for i in range(len(lv))]
vertex_to_tri = [set() for i in range(len(lv))]
corner_table = [[None]*5 for i in range(len(lt)*3)]    

In [84]:
def cn(idx):
    return corner_table[idx][2]
def cp(idx):
    return corner_table[idx][3]
def co(idx):
    return corner_table[idx][4]
def cv(idx):
    return corner_table[idx][0]
def ct(idx):
    return corner_table[idx][1]

# here assumes v1,v2,v3 are in ccw order

def update_opposite(v1,v2,v3,v1cidx):
    v1o = vertex_to_tri[v2]&vertex_to_tri[v3]
    if v1o:
        o_tri_idx = v1o.pop()
        o_c_idx = [o_tri_idx*3,o_tri_idx*3+1,o_tri_idx*3+2]
        o_corner_idx = None
        for c in o_c_idx:
            v = cv(c)
            if v in vertex_to_corner[v2] or c in vertex_to_corner[v3]:
                pass
            else:
                o_corner_idx = c
        
        corner_table[v1cidx][4] = o_corner_idx
        corner_table[o_corner_idx][4] = v1cidx
    
def insert_triangle(v1,v2,v3,idx):
    corner_table[idx*3] = [v1,idx,idx*3+1,idx*3+2,None]
    corner_table[idx*3+1] = [v2,idx,idx*3+2,idx*3,None]
    corner_table[idx*3+2] = [v3,idx,idx*3,idx*3+1,None]
    
    vertex_to_corner[v1].add(idx*3)
    vertex_to_corner[v2].add(idx*3+1)
    vertex_to_corner[v3].add(idx*3+2)
    
    update_opposite(v1,v2,v3,idx*3)
    update_opposite(v2,v3,v1,idx*3+1)
    update_opposite(v3,v1,v2,idx*3+2)
    
    vertex_to_tri[v1].add(idx)
    vertex_to_tri[v2].add(idx)
    vertex_to_tri[v3].add(idx)
    
for tidx, t in enumerate(lt):
    p1 = lv[t[0]]
    p2 = lv[t[1]]
    p3 = lv[t[2]]
    if ccw(p1,p2,p3):
        insert_triangle(t[0],t[1],t[2],tidx)
    else:
        insert_triangle(t[0],t[2],t[1],tidx)

In [87]:
def find_adjacent_vertice(vidx):
    result = set()
    for i in vertex_to_corner[vidx]:
        result.add(cv(cn(i)))
        result.add(cv(cp(i)))
    return result

def find_vertice_of_tri(tidx):
    return [corner_table[tidx*3][0],corner_table[tidx*3+1][0],corner_table[tidx*3+2][0]]

def find_tri_shard_vertice(tidx):
    result = []
    for i in range(3*tidx,3*tidx+3):
        if co(i):
            ocidx = co(i)
            result.append(ct(ocidx))
    return result
print('The coordinates of the vertices linked to the vertex with index 3:')
print(np.array(list(map(lambda x: lv[x],find_adjacent_vertice(3)))))
print('The coordinates of the vertices making up the face (triangle) with index 3:')
print(np.array(list(map(lambda x: lv[x],find_vertice_of_tri(3)))))
print('The coordinates of the vertices of the triangles sharing an edge with the triangle with index 3:')
for i in find_tri_shard_vertice(3):
    print(np.array(list(map(lambda x: lv[x],lt[i]))))

The coordinates of the vertices linked to the vertex with index 3:
[[-50.        -9.622504  25.      ]
 [-67.928932   7.071068  25.      ]
 [-67.928932  -7.071068  25.      ]
 [-65.         0.         8.333333]
 [-67.928932   7.071068   8.333333]
 [-50.         9.622504  25.      ]]
The coordinates of the vertices making up the face (triangle) with index 3:
[[-75.       -10.         8.333333]
 [-75.       -10.        25.      ]
 [-67.928932  -7.071068   8.333333]]
The coordinates of the vertices of the triangles sharing an edge with the triangle with index 3:
[[-75.       -10.         8.333333]
 [-67.928932  -7.071068   8.333333]
 [-67.928932  -7.071068  -8.333333]]
[[-75.       -10.        25.      ]
 [-67.928932  -7.071068  25.      ]
 [-67.928932  -7.071068   8.333333]]
